In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd


import random

random.seed(10)

N=9
nowcast=1

y_input_size=1
z_input_size=1

file="solartech_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2017-06-13 00:00:00', 0.0, 24.851, 0.0],
       [1, '2017-06-13 00:10:00', 0.0, 24.802, 0.0],
       [2, '2017-06-13 00:20:00', 0.0, 24.758, 0.0],
       ...,
       [17133, '2017-11-21 23:30:00', 0.0, 7.949, 0.0],
       [17134, '2017-11-21 23:40:00', 0.0, 7.861, 0.0],
       [17135, '2017-11-21 23:50:00', 0.0, 7.681, 0.0]], dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
24.851
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(17136,)
[0.008708218143262458 0.008691047699858978 0.00867562934251708 ...
 0.002785466420699098 0.0027546297060152985 0.0026915546077984362]
(17136,)
[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 ...
 [0.         0.00278547]
 [0.         0.00275463]
 [0.         0.00269155]]
(17136, 2)
(17136, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])

(17127, 9, 2)
(17127, 1)
[98.72755432]
[0.40682649]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.00870822]
 [0.         0.00869105]
 [0.         0.00867563]
 [0.         0.00867878]
 [0.         0.008649  ]
 [0.         0.00857646]
 [0.         0.00851269]
 [0.         0.00850287]
 [0.         0.00851304]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K


mae = tf.keras.losses.MeanAbsoluteError()
    

    

def make_generator():
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    noise = Input(shape=(z_input_size,))
    noise_output = Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.Zeros(), bias_initializer='zeros')(noise)

    concat = concatenate([latent, noise_output])
    concat = Dense(256)(concat)
    concat = Dense(128)(concat)
    y_fake = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x,noise], outputs=y_fake)
    return model  






def make_discriminator():
    
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    label = Input(shape=(y_input_size,))
    label_output = Dense(64, activation='relu')(label)

    concat = concatenate([latent, label_output])
    concat = Dense(256)(concat)
    concat = Dense(128)(concat)
    validity = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x, label], outputs=validity)
    model.compile(loss=['binary_crossentropy'],optimizer='adam')
    model.summary()
    return model  

x = Input(shape=(N,2))
noise = Input(shape=(z_input_size,))
generator = make_generator()
label_fake = generator([x,noise])


discriminator = make_discriminator()

validity = discriminator([x, label_fake])

discriminator.trainable = False

combined = Model([noise, x], [label_fake,validity])
combined.compile(loss=['binary_crossentropy'],optimizer='adam',metrics=['accuracy'])
#combined.compile(loss=loss_error_gan,optimizer='adam')
combined.summary()

#combined = Sequential()
#combined.add(generator)
#combined.add(lier)
#combined.add(discriminator)
#combined.compile(loss='binary_crossentropy', optimizer='adam')
    


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 9, 2)]       0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 8, 16)        80          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 6, 32)        1568        conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 4, 64)        6208        conv1d_4[0][0]                   
____________________________________________________________________________________________

In [8]:
print(x_train.shape)
print(y_train.shape)



#history = generator.fit(x_train,y_train ,
#                    batch_size=64,
#            validation_data=(x_val,y_val),
#            epochs=40)

(13701, 9, 2)
(13701, 1)


In [9]:
#from sklearn.metrics import confusion_matrix
#yhat = generator.predict(x_val)
#yhat=scaler.inverse_transform(yhat).flatten()
#y_val0=scaler.inverse_transform(y_val).flatten()
#print(y_val)
#print(yhat)

#file = open('prediction.csv', 'w')
#file.write('ground truth\tprediction\n')
#for i in range(0,len(y_val)):
#    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
#file.close()

#print("RMSE")
#error=tf.keras.metrics.mean_squared_error(
#    y_val0, yhat
#)
#print(np.sqrt(error))

In [10]:
def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels

def getNoise():
    return  np.random.normal(0, 1, (batch_size, z_input_size))



In [11]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

    
n_epochs=20
batch_size=64

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        
        
        noise = getNoise()
        x,true_labels = generate_true_samples(batch_size)
        fake_labels = generator.predict([x,noise])

        discriminator.train_on_batch([x, true_labels], valid+0.05 * tf.random.uniform(tf.shape(true_labels)))
        discriminator.train_on_batch([x, fake_labels], fake+0.05 * tf.random.uniform(tf.shape(fake_labels)))
        
        x,true_labels = generate_true_samples(batch_size)
        fake_labels = generator.predict([x,noise])

        combined.train_on_batch([noise,x], [true_labels,fake_labels])
        
        #d_loss_fake = discriminator.train_on_batch([x, predicted_labels_inc], inc_fake_labels)
        #d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        


        #lier.trainable = False
        #x2,_ = generate_true_samples(batch_size)
        #g_loss = combined.train_on_batch([noise,x], inc_fake_labels)
        #g_loss = combined.train_on_batch([noise,x], inc_fake_labels)
    
    #pf = discriminator.predict([x, predicted_labels])        
    #pt = discriminator.predict([x, predicted_labels_inc])
    #pc = combined.predict([noise,x2])

    #print('fake:',pf)    
    #print('true:',pt)    
    #print('combined:',pc)
    print(epoch, " of ", n_epochs)
    noise = getNoise()
    x,true_labels = generate_true_samples(batch_size)
    fake_labels = generator.predict([x,noise])
    print("true_labels:",true_labels)
    print("fake:",fake_labels)
    #print("g_loss:",g_loss)
    #print("d_loss:",d_loss)

e: 0
0  of  20
true_labels: [[0.64950805]
 [0.        ]
 [0.        ]
 [0.70089081]
 [0.        ]
 [0.65115476]
 [0.80633102]
 [0.        ]
 [0.68969452]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00220157]
 [0.28801946]
 [0.        ]
 [0.09062067]
 [0.60620829]
 [0.        ]
 [0.23760091]
 [0.        ]
 [0.06803692]
 [0.01801332]
 [0.        ]
 [0.07665376]
 [0.        ]
 [0.        ]
 [0.12732279]
 [0.        ]
 [0.24188601]
 [0.59163532]
 [0.        ]
 [0.        ]
 [0.47164674]
 [0.        ]
 [0.14604584]
 [0.68310938]
 [0.        ]
 [0.79988001]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00338619]
 [0.21073354]
 [0.68969452]
 [0.00672687]
 [0.        ]
 [0.38147742]
 [0.17399393]
 [0.        ]
 [0.0057878 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.64607777]
 [0.        ]
 [0.        ]
 [0.07400772]
 [0.385996  ]
 [0.        ]
 [0.        ]
 [0.        ]]
fake: [[0.58242893]
 [0.00622821]
 [0.00951049]
 [0.71040785]
 [0.00460014]

5  of  20
true_labels: [[0.        ]
 [0.0078142 ]
 [0.68146795]
 [0.58148926]
 [0.        ]
 [0.        ]
 [0.04402514]
 [0.        ]
 [0.0016519 ]
 [0.72198362]
 [0.35887849]
 [0.        ]
 [0.01150325]
 [0.        ]
 [0.        ]
 [0.82300368]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.17634805]
 [0.22587932]
 [0.        ]
 [0.03741352]
 [0.63935685]
 [0.0030407 ]
 [0.82844857]
 [0.47877986]
 [0.81860985]
 [0.42278511]
 [0.02877879]
 [0.29740606]
 [0.71211368]
 [0.        ]
 [0.83097323]
 [0.07958134]
 [0.        ]
 [0.00318003]
 [0.        ]
 [0.        ]
 [0.01737932]
 [0.        ]
 [0.76582602]
 [0.        ]
 [0.02189476]
 [0.67477628]
 [0.78516902]
 [0.53265212]
 [0.        ]
 [0.59329051]
 [0.        ]
 [0.35876232]
 [0.53969855]
 [0.        ]
 [0.34561011]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.03234894]
 [0.        ]
 [0.50543487]
 [0.8204003 ]
 [0.18626975]
 [0.00308185]]
fake: [[5.4633617e-04]
 [1.2527704e-03]
 [7.1145087e-01]
 [7.5732893e-01]
 [6.77

9  of  20
true_labels: [[0.        ]
 [0.70034555]
 [0.78032515]
 [0.        ]
 [0.81193915]
 [0.        ]
 [0.        ]
 [0.12619014]
 [0.34869118]
 [0.52780023]
 [0.00296917]
 [0.00301275]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.75207716]
 [0.        ]
 [0.10155734]
 [0.13802786]
 [0.52189868]
 [0.75288504]
 [0.58248781]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.89903945]
 [0.        ]
 [0.1448436 ]
 [0.00285059]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.76289782]
 [0.        ]
 [0.09559581]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.25940137]
 [0.33359931]
 [0.08903113]
 [0.2244948 ]
 [0.00247633]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.09678362]
 [0.56783174]
 [0.        ]
 [0.14604584]
 [0.78691786]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.12158612]
 [0.        ]
 [0.56404911]
 [0.72579095]
 [0.54480532]
 [0.66947953]
 [0.        ]]
fake: [[0.00400269]
 [0.73279727]
 [0.81303585]
 [0.00378504]
 [0.8329278 ]
 [0.

14  of  20
true_labels: [[0.08844162]
 [0.76977825]
 [0.        ]
 [0.        ]
 [0.04531989]
 [0.47394693]
 [0.        ]
 [0.38114789]
 [0.29511848]
 [0.634022  ]
 [0.47877986]
 [0.11442153]
 [0.78809398]
 [0.0882316 ]
 [0.22262752]
 [0.        ]
 [0.56215443]
 [0.03435907]
 [0.49788401]
 [0.80583352]
 [0.07446282]
 [0.83377793]
 [0.24132512]
 [0.60354508]
 [0.00298357]
 [0.1770006 ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.64331532]
 [0.01237372]
 [0.        ]
 [0.04232939]
 [0.05101802]
 [0.00734528]
 [0.06473296]
 [0.        ]
 [0.        ]
 [0.1872288 ]
 [0.6063661 ]
 [0.51011629]
 [0.        ]
 [0.66475298]
 [0.11683084]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.76111192]
 [0.        ]
 [0.        ]
 [0.62248614]
 [0.55454387]
 [0.        ]
 [0.37553181]
 [0.68929164]
 [0.        ]
 [0.11741912]
 [0.23544422]
 [0.00216776]
 [0.63516089]
 [0.42835234]
 [0.        ]
 [0.40404931]]
fake: [[6.48621619e-02]
 [7.32631266e-01]
 [1.80721283e-04]
 [1.96516514e-04]
 

19  of  20
true_labels: [[0.67330848]
 [0.        ]
 [0.3462265 ]
 [0.21198528]
 [0.00321188]
 [0.05235094]
 [0.13902612]
 [0.        ]
 [0.        ]
 [0.0920334 ]
 [0.        ]
 [0.        ]
 [0.06082341]
 [0.18541475]
 [0.        ]
 [0.66253861]
 [0.        ]
 [0.84200473]
 [0.35857271]
 [0.        ]
 [0.53030515]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.77934444]
 [0.        ]
 [0.63817174]
 [0.552222  ]
 [0.        ]
 [0.        ]
 [0.16036956]
 [0.00739388]
 [0.77901121]
 [0.74861553]
 [0.73808011]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.65030283]
 [0.18951377]
 [0.01457123]
 [0.00110663]
 [0.74850273]
 [0.61664343]
 [0.10598524]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.08044953]
 [0.00343257]
 [0.04864903]
 [0.01838955]
 [0.27419326]
 [0.        ]
 [0.00160232]
 [0.        ]
 [0.11990428]
 [0.        ]]
fake: [[6.6721094e-01]
 [4.9244089e-05]
 [3.4015030e-01]
 [2.2635812e-01]
 [4.9

In [12]:
from sklearn.metrics import confusion_matrix

noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
yhat = generator.predict([x_val,noise])
    
yhat=scaler.inverse_transform(yhat).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
print(y_val)
print(yhat)

file = open('prediction.csv', 'w')
file.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
file.close()

print("RMSE")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
print(np.sqrt(error))

[[0.0141186 ]
 [0.        ]
 [0.        ]
 ...
 [0.07686223]
 [0.        ]
 [0.81869012]]
[1.4730049e+00 7.2711315e-03 7.6513342e-03 ... 1.9491537e+01 1.7166352e-02
 1.7919495e+02]
RMSE
5.94084
